In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage import data, color, img_as_float
from skimage.filters import sobel
from scipy.interpolate import splprep, splev
from PIL import Image



# Load an image (can be any image, here using skimage's built-in data)
image = img_as_float(data.coins())
print(image.ndim)


# Apply Gaussian filter to smooth the image
smoothed_image = gaussian_filter(image, sigma=2)

plt.figure(figsize=(6, 6))
plt.imshow(image, cmap='gray')

# Compute edges using a Sobel filter (can use any gradient filter)
edges = sobel(smoothed_image)

# Plot original image and edge image
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original Image')
ax[1].imshow(edges, cmap='gray')
ax[1].set_title('Edge Image')
plt.show()
# Define initial snake contour as a circular or any arbitrary shape
t = np.linspace(0, 2*np.pi, 400)
x = 100 + 50*np.cos(t)  # X-coordinates of the snake
y = 150 + 50*np.sin(t)  # Y-coordinates of the snake

snake = np.array([x, y]).T

# Plot initial snake
plt.imshow(image, cmap='gray')
plt.plot(snake[:, 0], snake[:, 1], 'r-', lw=3)  # Plot the initial contour
plt.title("Initial Snake")
plt.show()
def evolve_snake(snake, alpha, beta, gamma, iterations, edge_map):
    for _ in range(iterations):
        # Compute internal forces (second derivative for smoothness)
        a = np.roll(snake, -1, axis=0)  # Next point
        b = np.roll(snake, 1, axis=0)   # Previous point
        internal_force = alpha * (b - 2*snake + a)

        # Compute external forces (based on image edges)
        fx = np.gradient(edge_map, axis=1)
        fy = np.gradient(edge_map, axis=0)

        external_force = np.array([fx[snake[:, 1].astype(int), snake[:, 0].astype(int)],
                                   fy[snake[:, 1].astype(int), snake[:, 0].astype(int)]]).T

        # Update snake position using a combination of forces
        snake += gamma * internal_force + beta * external_force

        # Apply spline smoothing to keep snake smooth
        tck, u = splprep([snake[:, 0], snake[:, 1]], s=2.0)
        snake = np.array(splev(u, tck)).T

    return snake

# Set parameters for snake evolution
alpha = 0.1  # Smoothness weight
beta = 1.0   # External (edge attraction) weight
gamma = 0.1  # Step size for evolution
iterations = 250

# Evolve the snake
final_snake = evolve_snake(snake, alpha, beta, gamma, iterations, edges)

# Plot the final snake
plt.imshow(image, cmap='gray')
plt.plot(final_snake[:, 0], final_snake[:, 1], 'r-', lw=3)  # Plot the evolved contour
plt.title("Final Snake")
plt.show()


2
